In [38]:
import os
from os import listdir
from os import path
from os.path import isfile, join, getsize
import re
import pandas as pd

import json
import io
import glob

#need this for some stupid reason to import essentia
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")
import essentia
import essentia.standard as es

from essentia.pytools.spectral import hpcpgram
%matplotlib inline
import matplotlib.pyplot as plt

In [39]:
ext_drive_dir = '/volumes/CCV_SSD_1/'
working_root_dir = 'covers32k/'
output = 'audio_output_stems/'
#png_folder = 'png_output/'
#harmonic_pitch_class_profile = 'harmonic_pitch_class_profile/'
#cover_song_simularity = 'cover_song_simularity/'
#cross_recurrent_plot = 'cross_recurrent_plot/'

#audio_stems = 'audio_output_stems/'

os.chdir(ext_drive_dir+working_root_dir)


#get all the directories for the songs
directories_songs = []
for x in os.walk(ext_drive_dir+working_root_dir):
    directories_songs.append(x[1])

directories_songs = directories_songs[0]
directories_songs.remove('audio_output_stems')
directories_songs.remove('pretrained_models')
directories_songs[:5]

['A_Whiter_Shade_Of_Pale',
 'Abracadabra',
 'Addicted_To_Love',
 'All_Along_The_Watchtower',
 'All_Tomorrow_s_Parties']

In [40]:
#get all the directories for the stems
directories_stems = []
for x in os.walk(ext_drive_dir+working_root_dir+output):
    directories_stems.append(x[1])

directories_stems = directories_stems[0]
directories_stems[:5]

['10cc+The_Very_Best_of_10cc+07-I_m_Not_In_Love',
 'a_ha+Hunting_High_and_Low+01-Take_On_Me',
 'aerosmith+A_Little_South_Of_Sanity_Disc_2_+08-Walk_This_Way',
 'aerosmith+Armageddon_Original_Soundtrack_+01-I_Don_t_Want_To_Miss_A_Thing',
 'aerosmith+Live_Bootleg+06-Come_Together']

In [41]:
filename_list1 = []
filestem_list1 = []
result = pd.DataFrame({'pair':[], 
                       'song_1':[],
                       'song_2':[],
                       'MFCC_1_bass':[], 
                       'MFCC_1_drums':[],
                       'MFCC_1_vocals':[],
                       'MFCC_1_other':[],
                       'GFCC_1_bass':[],
                       'GFCC_1_drums':[],
                       'GFCC_1_vocals':[],
                       'GFCC_1_other':[],
                       'HPCP_1_bass':[],
                       'HPCP_1_drums':[],
                       'HPCP_1_vocals':[],
                       'HPCP_1_other':[],
                       'MFCC_2_bass':[], 
                       'MFCC_2_drums':[],
                       'MFCC_2_vocals':[],
                       'MFCC_2_other':[],
                       'GFCC_2_bass':[],
                       'GFCC_2_drums':[],
                       'GFCC_2_vocals':[],
                       'GFCC_2_other':[],
                       'HPCP_2_bass':[],
                       'HPCP_2_drums':[],
                       'HPCP_2_vocals':[],
                       'HPCP_2_other':[],
                       'sim_matrix_bass':[],
                       'sim_matrix_drums':[],
                       'sim_matrix_vocals':[],
                       'sim_matrix_other':[],
                       'sim_distance_bass':[],
                       'sim_distance_drums':[],
                       'sim_distance_vocals':[],
                       'sim_distance_other':[],
                       'sim_distance_argmin':[]})

#set file names since they are all the same
bass = 'bass.wav'
drums = 'drums.wav'
voice = 'vocals.wav'
other = 'other.wav'

stems = (bass, drums, voice, other)

#look for mp3 and wav.
#the wav file here is the baseline... a guassian whitenoise at 0db

types = ('*.mp3', '*.wav')
#find all file pairs
for f in types:
    filename_list1.extend(glob.glob(f))
    
#split the filenames
for pair in directories_songs:
    for d in directories_stems:
        if path.exists(ext_drive_dir+working_root_dir+pair+'/'+d+'.mp3') == True:
            os.chdir(ext_drive_dir+working_root_dir+output+d)
            for f in glob.glob('*.wav'):
                filestem = f.split('.')
                filestem_list1.append([pair,d,filestem[0],filestem[1]])

filestem_list2 = filestem_list1
filestem_list1[:5]

[['A_Whiter_Shade_Of_Pale',
  'annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale',
  'bass',
  'wav'],
 ['A_Whiter_Shade_Of_Pale',
  'annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale',
  'drums',
  'wav'],
 ['A_Whiter_Shade_Of_Pale',
  'annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale',
  'other',
  'wav'],
 ['A_Whiter_Shade_Of_Pale',
  'annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale',
  'vocals',
  'wav'],
 ['A_Whiter_Shade_Of_Pale',
  'procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale',
  'bass',
  'wav']]

In [42]:
from essentia.standard import *

logNorm = UnaryOperator(type='log')

def extract_mfcc(audio_location):
    audio = es.MonoLoader(filename=audio_location, sampleRate=32000)()
    w = Windowing(type = 'hann')
    spectrum = Spectrum()
    mfcc = essentia.standard.MFCC()
    mfccs =[]
    melbands = []
    audio = essentia.array(audio)
    for frame in FrameGenerator(audio, frameSize = 1024 , hopSize = 512):
        mfcc_bands, mfcc_coeffs = mfcc(spectrum(w(frame)))
        mfccs.append(mfcc_coeffs)
        melbands.append(mfcc_bands)
    mfcc = essentia.array(melbands).T
    return mfcc

def extract_gfcc(audio_location):
    audio = es.MonoLoader(filename=audio_location, sampleRate=32000)()
    w = Windowing(type = 'hann')
    spectrum = Spectrum()
    gfcc = essentia.standard.GFCC()
    gfccs =[]
    gfccbands = []
    audio = essentia.array(audio)
    for frame in FrameGenerator(audio, frameSize = 1024 , hopSize = 512):
        gfcc_bands, gfcc_coeffs = gfcc(spectrum(w(frame)))
        gfccs.append(gfcc_coeffs)
        gfccbands.append(gfcc_bands)
    gfcc = essentia.array(gfccbands).T
    return gfcc

def extract_hpcp(audio_location):
    audio = es.MonoLoader(filename=audio_location, sampleRate=32000)()
    hpcp = hpcpgram(audio, sampleRate=32000)
    #hpcp = essentia.array(logNorm(hpcp)).T
    return hpcp

def extract_cover_similarity(f1_location, f2_location):
    #Calculate ChromaCrossSimularity
    song_1 = es.MonoLoader(filename=f1_location, sampleRate=32000)()
    song_2 = es.MonoLoader(filename=f2_location, sampleRate=32000)()
    
    song_1_hpcp = extract_hpcp(f1_location)
    song_2_hpcp = extract_hpcp(f2_location)
    crp = es.ChromaCrossSimilarity(frameStackSize=9, 
                                 frameStackStride=1, 
                                 binarizePercentile=0.095,
                                 oti=True)

    true_pair_crp = crp(song_1_hpcp, song_2_hpcp)
    
    #Calculate Distance with CSS
    score_matrix, distance = es.CoverSongSimilarity(disOnset=0.5, 
                                                  disExtension=0.5, 
                                                  alignmentType='chen17',
                                                  distanceType='asymmetric')(true_pair_crp)
    
    return score_matrix, distance, song_1_hpcp, song_2_hpcp

In [43]:
processed = []
for num1, i in enumerate(filestem_list1):
    for num2, j in enumerate(filestem_list2):
        if i[0] == j[0] and i[1] != j[1]:
            #find distance and aggregate 
            #find distance against baseline song and aggregate
            #dir1 = i[1] + '.' + i[2]
            #dir2 = j[1] + '.' + j[2]
            
            
            #print(dir2)
            argmin = 0
            distance_bass = 0
            distance_drums = 0
            distance_voice = 0
            distance_other = 0
            
            if i[0] not in processed:
                print(i[0])
                for f in stems:
                #f = i[2]+'.'+i[3]
                    filename1 = ext_drive_dir+working_root_dir+output+i[1] + '/' + f 
                    filename2 = ext_drive_dir+working_root_dir+output+j[1] + '/' + f
                    print(filename1)
                    print(filename2)
                    if f == 'bass.wav':
                        mfcc_1_bass = extract_mfcc(filename1)
                        mfcc_2_bass = extract_mfcc(filename2)

                        gfcc_1_bass = extract_gfcc(filename1)
                        gfcc_2_bass = extract_gfcc(filename2)

                        score_matrix_bass, distance_bass, song_1_hpcp_bass, song_2_hpcp_bass = extract_cover_similarity(filename1, filename2)

                    elif f == 'drums.wav':
                        mfcc_1_drums = extract_mfcc(filename1)
                        mfcc_2_drums = extract_mfcc(filename2)

                        gfcc_1_drums = extract_gfcc(filename1)
                        gfcc_2_drums = extract_gfcc(filename2)

                        score_matrix_drums, distance_drums, song_1_hpcp_drums, song_2_hpcp_drums = extract_cover_similarity(filename1, filename2)

                    elif f == 'vocals.wav':
                        mfcc_1_vocals = extract_mfcc(filename1)
                        mfcc_2_vocals = extract_mfcc(filename2)

                        gfcc_1_vocals = extract_gfcc(filename1)
                        gfcc_2_vocals = extract_gfcc(filename2)

                        score_matrix_vocals, distance_vocals, song_1_hpcp_vocals, song_2_hpcp_vocals = extract_cover_similarity(filename1, filename2)

                    else:
                        mfcc_1_other = extract_mfcc(filename1)
                        mfcc_2_other = extract_mfcc(filename2)

                        gfcc_1_other = extract_gfcc(filename1)
                        gfcc_2_other = extract_gfcc(filename2)

                        score_matrix_other, distance_other, song_1_hpcp_other, song_2_hpcp_other = extract_cover_similarity(filename1, filename2)



                argset = set([distance_bass,
                           distance_drums,
                           distance_vocals,
                           distance_other])

                argmin = min(argset)
                


                new_row = {'pair':i[0], 
                           'song_1':i[1],
                           'song_2':j[1],
                           'MFCC_1_bass':mfcc_1_bass, 
                           'MFCC_1_drums':mfcc_1_drums,
                           'MFCC_1_vocals':mfcc_1_vocals,
                           'MFCC_1_other':mfcc_1_other,
                           'GFCC_1_bass':gfcc_1_bass,
                           'GFCC_1_drums':gfcc_1_drums,
                           'GFCC_1_vocals':gfcc_1_vocals,
                           'GFCC_1_other':gfcc_1_other,
                           'HPCP_1_bass':song_1_hpcp_bass,
                           'HPCP_1_drums':song_1_hpcp_drums,
                           'HPCP_1_vocals':song_1_hpcp_vocals,
                           'HPCP_1_other':song_1_hpcp_other,
                           'MFCC_2_bass':mfcc_2_bass, 
                           'MFCC_2_drums':mfcc_2_drums,
                           'MFCC_2_vocals':mfcc_2_vocals,
                           'MFCC_2_other':mfcc_2_other,
                           'GFCC_2_bass':gfcc_2_bass,
                           'GFCC_2_drums':gfcc_2_drums,
                           'GFCC_2_vocals':gfcc_2_vocals,
                           'GFCC_2_other':gfcc_2_other,
                           'HPCP_2_bass':song_2_hpcp_bass,
                           'HPCP_2_drums':song_2_hpcp_drums,
                           'HPCP_2_vocals':song_2_hpcp_vocals,
                           'HPCP_2_other':song_2_hpcp_other,
                           'sim_matrix_bass':score_matrix_bass,
                           'sim_matrix_drums':score_matrix_drums,
                           'sim_matrix_vocals':score_matrix_vocals,
                           'sim_matrix_other':score_matrix_other,
                           'sim_distance_bass':distance_bass,
                           'sim_distance_drums':distance_drums,
                           'sim_distance_vocals':distance_vocals,
                           'sim_distance_other':distance_other,
                           'sim_distance_argmin':argmin}

                
                result = result.append(new_row, ignore_index=True)  
                processed.append(i[0])
    

A_Whiter_Shade_Of_Pale
/volumes/CCV_SSD_1/covers32k/audio_output_stems/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale/bass.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale/bass.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale/drums.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale/drums.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale/vocals.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale/vocals.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale/other.wav
/volumes/CCV_SSD_1/covers32k/audio_output_stems/procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale/other.wav
Abracadabra
/volumes/CCV_SSD_1/covers32k/audio_output_stems/steve_miller_band+Steve_Miller_Band_Live_+09-Abracadabra/bass

In [44]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
result

,pair,song_1,song_2,MFCC_1_bass,MFCC_1_drums,MFCC_1_vocals,MFCC_1_other,GFCC_1_bass,GFCC_1_drums,GFCC_1_vocals,GFCC_1_other,HPCP_1_bass,HPCP_1_drums,HPCP_1_vocals,HPCP_1_other,MFCC_2_bass,MFCC_2_drums,MFCC_2_vocals,MFCC_2_other,GFCC_2_bass,GFCC_2_drums,GFCC_2_vocals,GFCC_2_other,HPCP_2_bass,HPCP_2_drums,HPCP_2_vocals,HPCP_2_other,sim_matrix_bass,sim_matrix_drums,sim_matrix_vocals,sim_matrix_other,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale,procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.044063,0.037751,0.035935,0.022592,0.022592
1,Abracadabra,steve_miller_band+Steve_Miller_Band_Live_+09-A...,sugar_ray+14_59+11-Abracadabra,"[[5.4035194e-09, 1.0683395e-18, 0.0, 0.0, 0.0,...","[[9.842001e-05, 1.594722e-13, 4.281959e-13, 2....","[[5.0497794e-05, 1.5254655e-13, 5.655497e-13, ...","[[9.675984e-06, 2.35649e-13, 9.3634695e-14, 2....","[[4.250656e-07, 8.417094e-17, 0.0, 0.0, 0.0, 5...","[[0.0077416375, 1.4810904e-11, 2.8574804e-11, ...","[[0.0039726105, 1.45000244e-11, 5.1205897e-11,...","[[0.00076087203, 1.6600365e-11, 6.0230193e-12,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.99928254, 0.25534013, 0.0, 0.0, 0.01742810...","[[0.3015239, 0.8344343, 0.08064311, 0.0, 0.0, ...","[[0.5151143, 0.36861077, 0.19071248, 0.0, 0.17...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.033398,0.036090,0.023981,0.036515,0.023981
2,Addicted_To_Love,robert_palmer+Riptide+03-Addicted_To_Love,tina_turner+Tina_Live_In_Europe_CD_1_+09-Addic...,"[[4.2951465e-09, 1.3287877e-17, 0.0, 0.0, 0.0,...","[[2.0642759e-08, 2.4859647e-16, 0.0, 0.0, 0.0,...","[[4.6005716e-10, 5.3505835e-20, 0.0, 0.0, 0.0,...","[[3.1071858e-09, 6

In [45]:
result.to_csv('/volumes/CCV_SSD_1/similar_songs/cover80_song_pair_stem_data.csv', index = False)

In [46]:
result.to_pickle('/volumes/CCV_SSD_1/similar_songs/cover80_song_pair_stem_data.pkl')

In [47]:
newdf = pd.read_pickle('/volumes/CCV_SSD_1/similar_songs/cover80_song_pair_stem_data.pkl')

In [48]:
newdf.head()

,pair,song_1,song_2,MFCC_1_bass,MFCC_1_drums,MFCC_1_vocals,MFCC_1_other,GFCC_1_bass,GFCC_1_drums,GFCC_1_vocals,GFCC_1_other,HPCP_1_bass,HPCP_1_drums,HPCP_1_vocals,HPCP_1_other,MFCC_2_bass,MFCC_2_drums,MFCC_2_vocals,MFCC_2_other,GFCC_2_bass,GFCC_2_drums,GFCC_2_vocals,GFCC_2_other,HPCP_2_bass,HPCP_2_drums,HPCP_2_vocals,HPCP_2_other,sim_matrix_bass,sim_matrix_drums,sim_matrix_vocals,sim_matrix_other,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale,procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.044063,0.037751,0.035935,0.022592,0.022592
1,Abracadabra,steve_miller_band+Steve_Miller_Band_Live_+09-A...,sugar_ray+14_59+11-Abracadabra,"[[5.4035194e-09, 1.0683395e-18, 0.0, 0.0, 0.0,...","[[9.842001e-05, 1.594722e-13, 4.281959e-13, 2....","[[5.0497794e-05, 1.5254655e-13, 5.655497e-13, ...","[[9.675984e-06, 2.35649e-13, 9.3634695e-14, 2....","[[4.250656e-07, 8.417094e-17, 0.0, 0.0, 0.0, 5...","[[0.0077416375, 1.4810904e-11, 2.8574804e-11, ...","[[0.0039726105, 1.45000244e-11, 5.1205897e-11,...","[[0.00076087203, 1.6600365e-11, 6.0230193e-12,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.99928254, 0.25534013, 0.0, 0.0, 0.01742810...","[[0.3015239, 0.8344343, 0.08064311, 0.0, 0.0, ...","[[0.5151143, 0.36861077, 0.19071248, 0.0, 0.17...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.033398,0.036090,0.023981,0.036515,0.023981
2,Addicted_To_Love,robert_palmer+Riptide+03-Addicted_To_Love,tina_turner+Tina_Live_In_Europe_CD_1_+09-Addic...,"[[4.2951465e-09, 1.3287877e-17, 0.0, 0.0, 0.0,...","[[2.0642759e-08, 2.4859647e-16, 0.0, 0.0, 0.0,...","[[4.6005716e-10, 5.3505835e-20, 0.0, 0.0, 0.0,...","[[3.1071858e-09, 6